In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Flatten,Dense,Dropout

In [2]:
df = pd.read_csv("Copy of sonar data.csv", header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [3]:
df.isnull().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [4]:
df[60].value_counts()

M    111
R     97
Name: 60, dtype: int64

In [5]:
X = df.drop(60, axis = 1)
y = df[60]

In [6]:
y = pd.get_dummies(y, drop_first = True)
y.head()

,R
0,1
1,1
2,1
3,1
4,1


In [7]:
y.value_counts()

R
0    111
1     97
dtype: int64

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size  =0.25, random_state = 1)

In [9]:
model = Sequential()
model.add(Dense(60, input_shape = (60,), activation = "relu"))
model.add(Dense(30, activation = "relu"))
model.add(Dense(13, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))
model.compile(optimizer  = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.fit(X_train, y_train, epochs =100, batch_size = 8)

Epoch 1/100
20/20 [==============================] - 1s 3ms/step - loss: 0.6982 - accuracy: 0.5192
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6787 - accuracy: 0.6603
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6649 - accuracy: 0.6923
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6433 - accuracy: 0.7564
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6268 - accuracy: 0.7628
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6030 - accuracy: 0.7115
Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5737 - accuracy: 0.7500
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5425 - accuracy: 0.7885
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5127 - accuracy: 0.8077
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4995 - accuracy: 0.7756
Epoch 11/

In [10]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.7306 - accuracy: 0.7885


[0.730590283870697, 0.7884615659713745]

In [11]:
y_pred = model.predict(X_test).reshape(-1)
print(y_pred[:10])
# Since it is giving a number between 0 and 1 we should round it
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 4ms/step
[3.3808351e-04 9.7211236e-01 9.7816509e-01 5.7941591e-03 1.0000000e+00
 9.9998194e-01 3.5483215e-02 1.0000000e+00 8.1575140e-03 1.0000000e+00]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


Since the evaluation of out test set gave us 76% accuracy, lets print a classification report

In [12]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.93      0.82        27
           1       0.89      0.64      0.74        25

    accuracy                           0.79        52
   macro avg       0.81      0.78      0.78        52
weighted avg       0.81      0.79      0.78        52



Now we are going to retrain the model but this time with dropout layers and see if that will improve the accuracy of the test set

In [13]:
model1 = Sequential()
model1.add(Dense(60, input_shape = (60,), activation = "relu"))
model1.add(Dropout(0.5))
model1.add(Dense(30, activation = "relu"))
model1.add(Dropout(0.5))
model1.add(Dense(13, activation = "relu"))
model1.add(Dropout(0.5))
model1.add(Dense(1, activation = "sigmoid"))
model1.compile(optimizer  = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model1.fit(X_train, y_train, epochs =100, batch_size = 8)

Epoch 1/100
20/20 [==============================] - 1s 3ms/step - loss: 0.7128 - accuracy: 0.5000
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7197 - accuracy: 0.4744
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.7041 - accuracy: 0.5128
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.7033 - accuracy: 0.5321
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6814 - accuracy: 0.5385
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6996 - accuracy: 0.5513
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6751 - accuracy: 0.5513
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6908 - accuracy: 0.5641
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6706 - accuracy: 0.5897
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6801 - accuracy: 0.5449
Epoch 11/

As we can see the accuacy of the training set has decreased but that is okay what we really want is the accuracy of the test set

In [14]:
model1.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.4138 - accuracy: 0.7885


[0.4138124883174896, 0.7884615659713745]

In [15]:
y_pred1 = model1.predict(X_test).reshape(-1)
y_pred1 = np.round(y_pred1)
print(classification_report(y_test, y_pred1))

2/2 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           0       0.77      0.85      0.81        27
           1       0.82      0.72      0.77        25

    accuracy                           0.79        52
   macro avg       0.79      0.79      0.79        52
weighted avg       0.79      0.79      0.79        52

